In [1]:
from configparser import ConfigParser
import json
import requests
from bs4 import BeautifulSoup
from datetime import date
import time
import random
import pickle
import pandas as pd
import numpy as np
import re
import os

In [3]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [71]:
blackList=[]
if os.path.exists("word_blacklist.json"):
    with open("word_blacklist.json", "r") as f:
        data = json.load(f)
        self.blackList = data["blacklist"]
        
def inBlackList(text):
        for elm in self.blackList:
            serachResult = re.search(elm, text.lower())
            if serachResult is not None:
                print(f"found {elm} in blackList")
                print(text)
                return False
        return True

print(inBlackList("no relocation"))

NameError: name 'self' is not defined

In [68]:
class JobSearchLinkedIn:
    def __init__(self):
        self.wait_time = 3
        self.config = ConfigParser()
        self.config.read('config.ini')
        self.job_id_cache = self.loadJobCache()
        self.blackList = []
        self.searchParam = {}
        self.loadDefaultSearchParam()
        self.df = self.loadDataframe()
        self.driver = self.openBrowser()
        
    def loadJobCache(self):
        job_ids = {}
        if os.path.exists('job_id_cache.pickle'):
            with open('job_id_cache.pickle', 'rb') as f:
                # Pickle the dictionary
                job_ids = pickle.load(f)
        print(job_ids)
        return job_ids

    def loadDefaultSearchParam(self):
        if os.path.exists("search_param.json"):
            with open("search_param.json", "r") as f:
                data = json.load(f)
                self.searchParam = data
                
    def loadBlackList(self):
        if os.path.exists("word_blacklist.json"):
            with open("word_blacklist.json", "r") as f:
                data = json.load(f)
                self.blackList = data["blacklist"]

    def inBlackList(self, text):
        for elm in self.blackList:
            serachResult = re.search(elm, text.lower())
            if serachResult is not None:
                print(f"found {elm} in blackList")
                print(text)
                return False
        return True

    def loadDataframe(self):
        dataFrameEmpty = {
            'id': [],
            'title': [],
            'company': [],
            'location': [],
            'link': [],
            "retrieved_on": [],
            "salary": [],
            "applicants": [],
            "description": []
            
        }
        df = pd.DataFrame(dataFrameEmpty)
        if os.path.exists('job_search_database.csv'):
            df = pd.read_csv('job_search_database.csv')
        return df

    def openBrowser(self):
        options = Options()
        options.add_argument('--ignore-certificate-errors')
        return uc.Chrome(options)

    def login(self):
        self.driver.get("https://www.linkedin.com/login")
        time.sleep(self.wait_time)
        
        # Input Username and pass
        username = self.driver.find_element(By.ID, "username")
        username.send_keys(self.config['account']['username'])
            
        pword = self.driver.find_element(By.ID, "password")
        pword.send_keys(self.config['account']['password']) 
        time.sleep(self.wait_time)
            
        self.driver.find_element(By.XPATH, "//button[@type='submit']").click()
        time.sleep(self.wait_time)

    def getLinkedInSearchURL(self):
        if len(self.searchParam.keys()) <= 0:
            return "https://www.linkedin.com/jobs/?"
        return f"https://www.linkedin.com/jobs/search/?currentJobId={self.searchParam["currentJobId"]}&distance={self.searchParam['distance']}&f_E=2&f_WT=1%2C3%2C2&geoId={self.searchParam['geoID']}&keywords={self.searchParam['keywords']}&origin=JOB_COLLECTION_PAGE_KEYWORD_HISTORY&refresh=true"
    
    def testSoup(self):
        #self.driver.get(self.getLinkedInSearchURL())
        #time.sleep(self.wait_time)
        #items = self.driver.find_elements(By.TAG_NAME, "ul")
        items = self.driver.find_elements(By.CSS_SELECTOR, ".job-search-card")
        return items
            
    def getJobData(self):
        self.driver.get(self.getLinkedInSearchURL())
        time.sleep(self.wait_time)
        blackListedJobs = 0
        job_posts = []
        # get job Search base
        resultList = []
        #resultList = self.driver.find_elements(By.TAG_NAME, "li")
        resultList = self.driver.find_element(By.CSS_SELECTOR, ".scaffold-layout__list-container")
        #jobs = resultList.find_elements(By.CSS_SELECTOR, ".job-search-card")
        jobs = resultList.find_elements(By.TAG_NAME, "li")
        for job in jobs:
            job_id = job.get_attribute('data-reference-id')
            #print(job.text)
            if (self.inBlackList(job.text) and job_id not in self.job_id_cache):
                self.job_id_cache[job_id] = True
                jobTitleContainerList = job.find_elements(By.CSS_SELECTOR, ".job-card-list__title")
                jobMetaData = job.find_elements(By.CSS_SELECTOR, ".job-card-container__metadata-item")
                posting_link = ""
                salary = ""
                job_title = ""
                location = ""
                today = date.today()
                if len(jobMetaData) > 1:
                    location = jobMetaData[0].text
                if len(jobMetaData) > 2:
                    salary = jobMetaData[1].text
                if len(jobTitleContainerList) > 0:
                    posting_link = jobTitleContainerList[0].get_attribute('href')
                    job_title = jobTitleContainerList[0].text
                company_name = job.find_elements(By.CSS_SELECTOR, ".job-card-container__primary-description")
                #self.get_element_text_by_classname(job, "job-card-container__primary-description")
                if job_title != "":
                    job_posts.append([job_id, job_title, company_name, location, posting_link, today.strftime('%B %d, %Y'), salary])
            else:
                blackListedJobs += 1
        print(f"Found {len(jobs)} jobs, {blackListedJobs} were seen before or blacklisted")
        return job_posts

    def getJobDescriptions(self, listOfJobs):
        newJobList = []
        blackListedJobs = 0
        for idx in range(len(listOfJobs)):
            job = listOfJobs[idx]
            self.driver.get(job[4])
            #salary = self.get_element_text_by_classname(job, "job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight")
            #numApplicants = self.get_element_text_by_classname(job, ".num-applicants__caption")
            tvm_text = self.get_element_text_by_classname(self.driver, "tvm__text tvm__text--neutral")
            # click expand button
            self.driver.find_element(By.CSS_SELECTOR, ".jobs-description__footer-button").click()
            description = self.get_element_text_by_classname(self.driver, "jobs-description-content__text")
            #print(description)
            description.replace(",", " ")
            if (self.inBlackList(description[17:]) == False):
                newJobList.append(listOfJobs[idx] + [tvm_text] + [description[17:]])
            
            time.sleep(random.randint(0, 30))
            
        print(f"Checked {len(listOfJobs)} jobs, {blackListedJobs} were blacklisted")
        
        for job in newJobList:
            self.df.loc[len(self.df.index)] = job

    def get_element_text_by_classname(self, documentObject, class_name):
        found = documentObject.find_elements(By.CSS_SELECTOR, "." + class_name)
        if len(found) <= 0:
            return ""
        return found[0].text

    def saveDictionary(name, dict):
        with open(name, 'wb') as f:
            pickle.dump(dict, f)

    def save(self):
        self.df.to_csv("job_search_database.csv", sep=',', index=False, encoding='utf-8')
        self.saveDictionary("job_id_cache.pickle", self.job_id_cache)

In [69]:
def main():
    jobSearch = JobSearchLinkedIn()
    jobSearch.login()
    jobData = jobSearch.getJobData()
    print(jobData)
    jobsOnPageWithDescription = jobSearch.getJobDescriptions(jobData)
    print(jobSearch.df)

    #jobSearch.save()

In [70]:
main()

{}
Found 49 jobs, 48 were seen before or blacklisted
[[None, 'Rails Developer', [<undetected_chromedriver.webelement.WebElement (session="8b21290f8221685efeee06be63447233", element="f.5A31F155714E5F081296C4828BE294E7.d.869AC6F1BEE13F0B43962FEA04E8F660.e.283")>], 'Denver, CO (Hybrid)', 'https://www.linkedin.com/jobs/view/3833818124/?eBP=CwEAAAGOWVsOOncoCSzyiW_FrlHK4FYH5bUmvIYYtZsq4tS4TajSk7BnVy1HAKSCytdmLIpa9W3Tea5UG-HWZJJHLz66rqRt3iSCsnIl512Zu5Dxx0JfEeIxS4_3sdUW7ZpaVTVnLcSXTti0DixvSk1gmPPJT7od0yyeZyG_JV69M78fSaJKlV6abHTyhHw4JGB-edkwEis5Dj2NZD4BsWA8affz7-ccFKnYLaYlILnK_E3k25FHutK1kG1LTR5oyUjQXyMHB2bGKwE-8D6t-752We-HbS8Kbp4v1F6zGqAeRrQiANmr5eUAJGDe0e7d8TOsX8-2zaf77FEeZxZLYONVd_vyImS2Gfi4DtQneRSJy8yLO4_npItg_0veJgih6so2n-jzbJjDmBwhlYQUrwB0HB6w&refId=nCCd4bIbgfGdtzSua4Cmow%3D%3D&trackingId=8TS2SPe3jGadQVrDEqPVLQ%3D%3D&trk=flagship3_search_srp_jobs', 'March 19, 2024', '']]
Checked 1 jobs, 0 were blacklisted
           id                                          title  \
0  3841120639       

In [43]:
jobSearch = JobSearchLinkedIn()

In [44]:
jobSearch.login()

In [45]:
jobData = jobSearch.getJobData()

Found 64 jobs, 63 were seen before or blacklisted


In [46]:
print(jobData)

[[None, 'Software Developer', 'Rise Broadband', 'Englewood, CO (On-site)', 'https://www.linkedin.com/jobs/view/3813296475/?eBP=CwEAAAGOWGs1NDbFxkmXsecy5oD6JinQghKFfUbrElgrF8b62YeBNLQplJrDBrBFm-TL2UCAGb9M_MA_jvcUwAeq2Qfh1I7KE3b6C8f4FVeb8cuAAiN2ayVbJT0dEzQyfyG4JU7Ob4UXrM3wQ3HPfAfk4iyUNHiL21XsGNyNbXFfNOT9APj90rtu3BI1_WFlz68HVuhDGbk20KRmOScjBRJfrsjtca8BuI23G8fW_sUkQ28kEf9yL-zfLn3nM0BCiUTeQ7rRfGNqdnl7iU7bh-CPDjmVIRqUNsNkpmxTGVl1WUhtKTKfgMDitCm_bgeyga3-evFwS0S_Ga7EPuN1O4oHpuYipqUwxRp6Ut2aAln1CM7Mb0kNFyy0gUfGcoblsO1LcGlcAlM6Z9ddZQxJR3uNkkzk&refId=Z0lv4EPsoOMziA0HmfL2Vg%3D%3D&trackingId=jOES6HJz2dyhbQ%2Fm3D3TRw%3D%3D&trk=flagship3_search_srp_jobs', 'March 19, 2024']]


In [47]:
jobsOnPageWithDescription = jobSearch.getJobDescriptions(jobData)

AttributeError: 'list' object has no attribute 'find_elements'

In [32]:
print(jobSearch.df)

           id                                          title  \
0  3841120639          Software Development Engineer in Test   
1  3822054670                                 Java Developer   
2  3845777305                       Ground Software Engineer   
3  3831952564   Software Engineer 2 (Java, Spring Boot, AWS)   
4  3830734256        Software Engineer 2 (Java, Spring Boot)   
5  3823523203                                  AEM Developer   
6  3730488628  Technical Lead, Software Development (Cerner)   

                     company                 location  salary  \
0             Charles Schwab  Lone Tree, CO (On-site)     NaN   
1  Tata Consultancy Services                      NaN     NaN   
2                   Actalent                      NaN     NaN   
3                  U.S. Bank               Denver, CO     NaN   
4                  U.S. Bank                      NaN     NaN   
5  Tata Consultancy Services                      NaN     NaN   
6         DaVita Kidney Care    

In [ ]:
jobSearch.save()

In [ ]:
testList = jobSearch.testSoup()

In [ ]:
for val in testList:
    print(val.get_attribute("class"))
    print("\n")